# Efectos de la Compresión de Modelos para Sistemas de Recomendación en Dispositivos Móviles

In [1]:
import cornac
import torch.ao.quantization
import recommenders
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.models.cornac.cornac_utils import predict_ranking


from recommenders.utils.timer import Timer

from src.compressors import quantization
from src.metrics import get_model_size, calculate_metrics

## Movielens

In [2]:

# MOVIELENS_DATA_SIZE = '100k'
MOVIELENS_DATA_SIZE = '1m'

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

100%|██████████| 5.78k/5.78k [00:01<00:00, 5.65kKB/s]


In [4]:
train, test = python_random_split(data, 0.75)

In [5]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 6040
Number of items: 3676


Model definition and training 

In [6]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    use_gpu=torch.cuda.is_available(),
    # use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 66.0911 seconds for training.


Metrics Calculation (NDCG and Recall)

In [7]:
from src.metrics import get_model_size, calculate_metrics

base_metrics = calculate_metrics(train,test,bivae)
print(base_metrics)

{'NDCG': 0.4136278628326316, 'Recall@K': 0.13876637955697035, 'Execution_time': 0.0010799661999044475}


In [ ]:
bivae.save('ckpts')

Quantization method

In [8]:
model_quatized = torch.ao.quantization.quantize_dynamic(
    bivae.bivae,
    {torch.nn.Linear},
    dtype=torch.qint8
)

In [9]:
quantized_bivae = quantization.get_post_dynamic(bivae)
quantized_bivae.train_set = train_set

calculate_metrics(train,test,quantized_bivae)

{'NDCG': 0.4136278628326316,
 'Recall@K': 0.13876637955697035,
 'Execution_time': 0.0007396696002009169}

In [ ]:
quantized_bivae.save("ckpts")

In [10]:
original = get_model_size(bivae.bivae, "original")
quantized = get_model_size(model_quatized,"quantized")

print(f"El tamaño del modelo reducido representa un: {(100*(quantized/original))}% del modelo original")

model:  original  	 Size (KB): 8099.607
model:  quantized  	 Size (KB): 2034.063
El tamaño del modelo reducido representa un: 25.11310733965241% del modelo original


Prunning method

Prunning + Quantization method

## Amazon reviews dataset 
### Clothing

In [3]:
import os
from urllib.request import urlopen

if not os.path.exists('./datasets/Clothing_Shoes_and_Jewelry.json.gz'):
    print("Dont exists!")
    !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Clothing_Shoes_and_Jewelry.json.gz
    os.rename('./Clothing_Shoes_and_Jewelry.json.gz', './datasets/Clothing_Shoes_and_Jewelry.json.gz')


In [2]:
# Dataset size (Dataset size limit due to hardware limitations)
DATASET_SIZE = int(0.75e6)

# Min interaction per user
MIN_RATES = 3

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [3]:
import json
import gzip
import pandas as pd
import hashlib

data = []
test_users = []

with gzip.open('datasets/Clothing_Shoes_and_Jewelry.json.gz') as f:
    for l in f:
        d = json.loads(l.strip())
        # asgining unique id to each user and item
        user = int.from_bytes(hashlib.sha256(d['reviewerID'].encode('utf-8')).digest()[:4], 'little')
        item = int.from_bytes(hashlib.sha256(d['asin'].encode('utf-8')).digest()[:4], 'little')
        rate = d['overall']

        entry = {'userID': user, 'itemID': item, 'rating': rate}

        data.append(entry)

        if len(data) >= DATASET_SIZE:
            break

data_set = pd.DataFrame.from_dict(data)

# Filter users with minimal interaction required
for u, n in data_set['userID'].value_counts().items():
    if n >= MIN_RATES:
        test_users.append(u)
        
data_set = data_set[data_set['userID'].isin(test_users)]

del data

In [4]:
train, test = python_random_split(data_set, 0.75)

In [5]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 22513
Number of items: 2410


/home/erick/Documents/Codigos/MC_6101/proyecto/env/lib/python3.7/site-packages/cornac/data/dataset.py:361: UserWarning: 1136 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


Model definition and training 

In [6]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    use_gpu=torch.cuda.is_available(),
    # use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 144.2751 seconds for training.


Metrics Calculation (NDCG and Recall)

In [7]:
from src.metrics import get_model_size, calculate_metrics

base_metrics = calculate_metrics(train,test,bivae)
print(base_metrics)

{'NDCG': 0.535461645259503, 'Recall@K': 0.5675428248420948}


Quantization method

In [8]:
model_quatized = torch.ao.quantization.quantize_dynamic(
    bivae.bivae,
    {torch.nn.Linear},
    dtype=torch.qint8
)

In [9]:
quantized_bivae = quantization.get_post_dynamic(bivae)
quantized_bivae.train_set = train_set

calculate_metrics(train,test,quantized_bivae)

{'NDCG': 0.535461645259503, 'Recall@K': 0.5675428248420948}

In [10]:
original = get_model_size(bivae.bivae, "original")
quantized = get_model_size(model_quatized,"quantized")

print(f"El tamaño del modelo reducido representa un: {(100*(quantized/original))}% del modelo original")

model:  original  	 Size (KB): 20265.239
model:  quantized  	 Size (KB): 5075.535
El tamaño del modelo reducido representa un: 25.045522532450764% del modelo original


Prunning method

Prunning + Quantization method

### Office

In [13]:
import os
from urllib.request import urlopen

if not os.path.exists('./datasets/Office_Products.json.gz'):
    !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Office_Products.json.gz
    os.rename('./Office_Products.json.gz', './datasets/Office_Products.json.gz')

In [3]:
# Dataset size (Dataset size limit due to hardware limitations)
DATASET_SIZE = int(0.5e6)

# Min interaction per user
MIN_RATES = 3

# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [6]:
import json
import gzip
import pandas as pd
import hashlib

data = []
test_users = []

with gzip.open('datasets/Office_Products.json.gz') as f:
    for l in f:
        d = json.loads(l.strip())
        # asgining unique id to each user and item
        user = int.from_bytes(hashlib.sha256(d['reviewerID'].encode('utf-8')).digest()[:4], 'little')
        item = int.from_bytes(hashlib.sha256(d['asin'].encode('utf-8')).digest()[:4], 'little')
        rate = d['overall']

        entry = {'userID': user, 'itemID': item, 'rating': rate}

        data.append(entry)

        if len(data) >= DATASET_SIZE:
            break

data_set = pd.DataFrame.from_dict(data)

# Filter users with minimal interaction required
for u, n in data_set['userID'].value_counts().items():
    if n >= MIN_RATES:
        test_users.append(u)
        
data_set = data_set[data_set['userID'].isin(test_users)]

del data

In [7]:
train, test = python_random_split(data_set, 0.75)

In [8]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 15899
Number of items: 3088


/home/erick/Documents/Codigos/MC_6101/proyecto/env/lib/python3.7/site-packages/cornac/data/dataset.py:361: UserWarning: 1649 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


Model definition and training

In [9]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    use_gpu=torch.cuda.is_available(),
    # use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 143.0167 seconds for training.


Metrics Calculation (NDCG and Recall)

In [10]:
from src.metrics import get_model_size, calculate_metrics

base_metrics = calculate_metrics(train,test,bivae)
print(base_metrics)

{'NDCG': 0.16503002124814123, 'Recall@K': 0.20795829115668152}


In [ ]:
bivae.save('ckpts')

Quantization method

In [15]:
model_quatized = torch.ao.quantization.quantize_dynamic(
    bivae.bivae,
    {torch.nn.Linear},
    dtype=torch.qint8
)

In [17]:
quantized_bivae = quantization.get_post_dynamic(bivae)
quantized_bivae.train_set = train_set

calculate_metrics(train,test,quantized_bivae)

{'NDCG': 0.16503002124814123, 'Recall@K': 0.20795829115668152}

In [ ]:
quantized_bivae.save("ckpts")

In [18]:
original = get_model_size(bivae.bivae, "original")
quantized = get_model_size(model_quatized,"quantized")

print(f"El tamaño del modelo reducido representa un: {(100*(quantized/original))}% del modelo original")

model:  original  	 Size (KB): 15516.439
model:  quantized  	 Size (KB): 3888.271
El tamaño del modelo reducido representa un: 25.059042219674243% del modelo original


Prunning method

Prunning + Quantization method

### Sports

In [2]:
import os
from urllib.request import urlopen

if not os.path.exists('./datasets/Sports_and_Outdoors.json.gz'):
    !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Sports_and_Outdoors.json.gz
    os.rename('./Sports_and_Outdoors.json.gz', './datasets/Sports_and_Outdoors.json.gz')

In [24]:
# Dataset size (Dataset size limit due to hardware limitations)
DATASET_SIZE = int(0.5e6)

# Min interaction per user
MIN_RATES = 3

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [25]:
import json
import gzip
import pandas as pd
import hashlib

data = []
test_users = []

with gzip.open('datasets/Sports_and_Outdoors.json.gz') as f:
    for l in f:
        d = json.loads(l.strip())
        # asgining unique id to each user and item
        user = int.from_bytes(hashlib.sha256(d['reviewerID'].encode('utf-8')).digest()[:4], 'little')
        item = int.from_bytes(hashlib.sha256(d['asin'].encode('utf-8')).digest()[:4], 'little')
        rate = d['overall']

        entry = {'userID': user, 'itemID': item, 'rating': rate}

        data.append(entry)

        if len(data) >= DATASET_SIZE:
            break

data_set = pd.DataFrame.from_dict(data)

# Filter users with minimal interaction required
for u, n in data_set['userID'].value_counts().items():
    if n >= MIN_RATES:
        test_users.append(u)
        
data_set = data_set[data_set['userID'].isin(test_users)]

del data

In [26]:
train, test = python_random_split(data_set, 0.75)

In [27]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 8600
Number of items: 2950


/home/erick/Documents/Codigos/MC_6101/proyecto/env/lib/python3.7/site-packages/cornac/data/dataset.py:361: UserWarning: 500 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


Model definition and training

In [29]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    use_gpu=torch.cuda.is_available(),
    # use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 63.2633 seconds for training.


In [9]:
bivae.save('ckpts')

BiVAECF model is saved to ckpts/BiVAECF/2023-05-30_18-46-33-347004.pkl


'ckpts/BiVAECF/2023-05-30_18-46-33-347004.pkl'

In [30]:
from src.metrics import get_model_size, calculate_metrics

base_metrics = calculate_metrics(train,test,bivae)
print(base_metrics)

{'NDCG': 0.12023943346502655, 'Recall@K': 0.1555377727567522}


Quantization method

In [31]:
model_quatized = torch.ao.quantization.quantize_dynamic(
    bivae.bivae,
    {torch.nn.Linear},
    dtype=torch.qint8
)

In [32]:
quantized_bivae = quantization.get_post_dynamic(bivae)
quantized_bivae.train_set = train_set

calculate_metrics(train,test,quantized_bivae)

{'NDCG': 0.12023943346502655, 'Recall@K': 0.1555377727567522}

In [33]:
original = get_model_size(bivae.bivae, "original")
quantized = get_model_size(model_quatized,"quantized")

print(f"El tamaño del modelo reducido representa un: {(100*(quantized/original))}% del modelo original")

model:  original  	 Size (KB): 9566.807
model:  quantized  	 Size (KB): 2400.847
El tamaño del modelo reducido representa un: 25.095593545474472% del modelo original


Prunning method

Prunning + Quantization method

## Epinions

In [40]:
import os
from urllib.request import urlopen

if not os.path.exists('./datasets/soc-Epinions1.txt.gz'):
    !wget https://snap.stanford.edu/data/soc-Epinions1.txt.gz
    os.rename('./soc-Epinions1.txt.gz', './datasets/soc-Epinions1.txt.gz')


In [11]:
# Dataset size (Dataset size limit due to hardware limitations)
DATASET_SIZE = int(1.5e5)

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [12]:
import json
import gzip
import pandas as pd
import hashlib

data = []

with gzip.open('datasets/soc-Epinions1.txt.gz') as f:
    for i, l in enumerate(f):
        if i>3:

            d = l.decode().split()
            entry = {'userID': int(d[0]), 'itemID': int(d[1]), 'rating': 5}
            data.append(entry)

        if len(data) >= DATASET_SIZE:
            break


data_set = pd.DataFrame.from_dict(data)

del data

In [13]:
train, test = python_random_split(data_set, 0.75)

In [14]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 2000
Number of items: 14015


Model definition and training

In [16]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    use_gpu=torch.cuda.is_available(),
    # use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 78.9307 seconds for training.


In [ ]:
bivae.save('ckpts')

In [17]:
from src.metrics import get_model_size, calculate_metrics

base_metrics = calculate_metrics(train,test,bivae)
print(base_metrics)

{'NDCG': 0.10900177312224939, 'Recall@K': 0.0673371572161086}


Quantization method

In [18]:
model_quatized = torch.ao.quantization.quantize_dynamic(
    bivae.bivae,
    {torch.nn.Linear},
    dtype=torch.qint8
)

In [19]:
quantized_bivae = quantization.get_post_dynamic(bivae)
quantized_bivae.train_set = train_set

calculate_metrics(train,test,quantized_bivae)

{'NDCG': 0.10900177312224939, 'Recall@K': 0.0673371572161086}

In [20]:
original = get_model_size(bivae.bivae, "original")
quantized = get_model_size(model_quatized,"quantized")

print(f"El tamaño del modelo quatizado representa un: {(100*(quantized/original))}% del modelo original")

model:  original  	 Size (KB): 13138.839
model:  quantized  	 Size (KB): 3293.839
El tamaño del modelo quatizado representa un: 25.06948292767725% del modelo original


Prunning method

Prunning + Quantization method